This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to Euclidean splines](index.ipynb)

# Cubic Quadrangle Interpolation

This doesn't seem to be a very popular type of spline.
We are mainly mentioning it because it is
the starting point for interpolating rotations with
[Spherical Quadrangle Interpolation (Squad)](../rotation/squad.ipynb).

In [ ]:
import sympy as sp
sp.init_printing(order='grevlex')

As usual, we import some helpers from
[utility.py](utility.py) and [helper.py](helper.py):

In [ ]:
from utility import NamedExpression, NamedMatrix
from helper import plot_basis

We start by looking at the fifth segment of a splines,
between $\boldsymbol{x}_4$ and $\boldsymbol{x}_5$:

In [ ]:
x4, x5 = sp.symbols('xbm4:6')

<cite data-cite="boehm1982cubics">Boehm (1982)</cite>
mentions (on page 203) so-called
*quadrangle points*:

In [ ]:
x4bar = sp.symbols('xbarbm4^(+)')
x5bar = sp.symbols('xbarbm5^(-)')
x4bar, x5bar

In [ ]:
t = sp.symbols('t')

In [ ]:
def lerp(one, two, t):
    """Linear intERPolation.

    The parameter *t* is expected to be between 0 and 1.

    """
    return (1 - t) * one + t * two

<cite data-cite="boehm1982cubics">Boehm (1982)</cite>
also mentions (on page 210)
a peculiar algorithm
to construct the spline segment.
In a first step,
a linear interpolation between the start and end point is done,
as well as a linear interpolation between the two quadrangle points.
The two resulting points are then interpolated again in a second step.
However, the last interpolation does not happen along a straight line,
but along a parabola defined by the expression $2t(1-t)$:

In [ ]:
p4 = NamedExpression(
    'pbm4',
    lerp(lerp(x4, x5, t), lerp(x4bar, x5bar, t), 2 * t * (1 - t)))

This leads to a cubic polynomial.
The following steps are very similar
to what we did for
[cubic Bézier curves](bezier-de-casteljau.ipynb#Degree-3-(Cubic)).

## Basis Polynomials

In [ ]:
b = [p4.expr.expand().coeff(x) for x in (x4, x4bar, x5bar, x5)]
b

In [ ]:
plot_basis(*b, labels=(x4, x4bar, x5bar, x5))

## Basis Matrix

In [ ]:
M_Q = NamedMatrix(
    r'{M_\text{Q}}',
    sp.Matrix([[c.coeff(x) for x in (x4, x4bar, x5bar, x5)]
               for c in p4.as_poly(t).all_coeffs()]))
M_Q

In [ ]:
M_Q.I

## Tangent Vectors

In [ ]:
pd4 = p4.diff(t)

In [ ]:
xd4 = pd4.evaluated_at(t, 0).with_name('xdotbm4^(+)')
xd4

In [ ]:
xd5 = pd4.evaluated_at(t, 1).with_name('xdotbm5^(-)')
xd5

## Quadrangle to Hermite Control Values

In [ ]:
M_QtoH = NamedMatrix(
    r'{M_\text{Q$\to$H}}',
    sp.Matrix([[expr.coeff(cv) for cv in [x4, x4bar, x5bar, x5]]
               for expr in [
                   x4,
                   x5,
                   xd4.expr,
                   xd5.expr]]))
M_QtoH

In [ ]:
M_QtoH.I.pull_out(sp.S.One / 2)

## Quadrangle to Bézier Control Points

Since we already know the tangent vectors,
it is easy to find the Bézier control points,
as we have already shown in the notebook about
[uniform Hermite splines](hermite-uniform.ipynb#Relation-to-Bézier-Splines).

In [ ]:
x4tilde = NamedExpression('xtildebm4^(+)', x4 + xd4.expr / 3)
x4tilde

In [ ]:
x5tilde = NamedExpression('xtildebm5^(-)', x5 - xd5.expr / 3)
x5tilde

In [ ]:
M_QtoB = NamedMatrix(
    r'{M_\text{Q$\to$B}}',
    sp.Matrix([[expr.coeff(cv) for cv in (x4, x4bar, x5bar, x5)]
               for expr in [
                   x4,
                   x4tilde.expr,
                   x5tilde.expr,
                   x5]]))
M_QtoB.pull_out(sp.S.One / 3)

In [ ]:
M_QtoB.I.pull_out(sp.S.One / 2)

The inverse matrix can be used for converting
from Bézier control points to quadrangle points:

In [ ]:
NamedMatrix(
    sp.Matrix([x4, x4bar, x5bar, x5]),
    M_QtoB.I.expr * sp.Matrix([x4, x4tilde.name, x5tilde.name, x5]))

We can generalize the equations
for the outgoing and incoming quadrangle points:

\begin{align*}
\boldsymbol{\bar{x}}_i^{(+)} &=
\frac{3}{2} \boldsymbol{\tilde{x}}_i^{(+)} -
\frac{1}{2} \boldsymbol{x}_{i+1}\\
\boldsymbol{\bar{x}}_i^{(-)} &=
\frac{3}{2} \boldsymbol{\tilde{x}}_i^{(-)} -
\frac{1}{2} \boldsymbol{x}_{i-1}
\end{align*}

The two equations are also shown in
<cite data-cite="boehm1982cubics">Boehm (1982)</cite>
on page 203.

## Non-Uniform Parameterization

Just like
[cubic Bézier splines](bezier-non-uniform.ipynb),
the shape of a segment (i.e. the *image*) is fully defined by its four control points.
Re-scaling the parameter does not change the shape,
but it changes the speed and therefore the tangent vectors.

In [ ]:
t4, t5 = sp.symbols('t4:6')

In [ ]:
p4nu = p4.subs(t, (t - t4) / (t5 - t4)).with_name(
    r'\boldsymbol{p}_\text{4,non-uniform}')

In [ ]:
pd4nu = p4nu.diff(t)

In [ ]:
pd4nu.evaluated_at(t, t4)

In [ ]:
pd4nu.evaluated_at(t, t5)